In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv('part_1.csv')

In [2]:
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,y_class
0,2.7,-4.55,6.4,3.7,5,1,0,-15.55,0
1,-2.2,0.05,9.7,19.5,-56,0,1,-16.45,1
2,7.2,3.05,-9.8,-13.5,35,1,1,20.75,0
3,6.7,1.30,9.7,22.0,-7,1,1,14.00,0
4,7.1,-4.10,-6.8,-21.8,-45,0,1,-1.00,0


In [3]:
df.describe()

,x1,x2,x3,x4,x5,x6,x7,x8,y_class
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.00000,400.000000,400.000000,400.000000
mean,0.073500,-0.014875,0.434250,0.838750,-0.777500,0.50750,0.530000,-9.824125,0.445000
std,5.707309,2.836474,5.736643,12.414111,59.433904,0.50057,0.499724,18.677460,0.497588
min,-10.000000,-5.000000,-9.900000,-27.800000,-100.000000,0.00000,0.000000,-48.250000,0.000000
25%,-4.700000,-2.300000,-4.225000,-8.325000,-52.500000,0.00000,0.000000,-24.212500,0.000000
50%,-0.150000,-0.175000,0.850000,1.800000,3.500000,1.00000,1.000000,-8.350000,0.000000
75%,5.200000,2.312500,5.450000,10.200000,47.250000,1.00000,1.000000,2.787500,1.000000
max,9.900000,4.950000,9.900000,29.000000,99.000000,1.00000,1.000000,32.150000,1.000000


In [4]:
df.x1.nunique()

173

In [5]:
df.x2.nunique()

179

In [6]:
df.x3.nunique()

174

In [7]:
df.x4.nunique()

266

In [8]:
df.x5.nunique()

169

In [9]:
df.x6.nunique()

2

In [10]:
df.x7.nunique()

2

In [11]:
df.x8.nunique()

266

In [12]:
df.y_class.nunique()

2

In [13]:
df.isnull().any()

x1         False
x2         False
x3         False
x4         False
x5         False
x6         False
x7         False
x8         False
y_class    False
dtype: bool

All the columns except x6, x7 and y_class have a large number of unique values which could cause a classifier like decision tree to overfit. 
I can either amend my data to prevent this or choose a more suitable classifier.

I first toyed with the idea of categorizing my data (columns x1-x5 and x8) to lessen the number of unique values into more of a series of intervals (say,5). I even created the following function to create classes out of intervals.

In [14]:
#def f(row):
#    if -10<=row['x1']<-6:
#        val = 0
#    elif -6<=row['x1']<-2:
#        val = 1
#    elif -2<=row['x1']<2:
#        val = 2
#    elif 2<=row['x1']<6:
#        val=3
#    elif 6<=row['x1']<10:
#        val=4   
#    return val

#df['x1'] = df.apply(f, axis=1)

After running the rest of my code with this function, testing it with multiple classifiers and comparing results with and without the use of this function, I decided NOT use it.
The most significant difference of using and not using this function was that the score of the KNearest neighbor classifier went from 0.7 to 0.55, respectively. Decision Tree classifier score also went from 0.8 to 0.725.

This result was expected as these models are subject to the risk of overfitting.
My initial intention was to use an ensemble classifier, containing 3 different classifiers. Specifically, the best performing of decision tree, KNeighbors, Random Forests and Logistic Regression.
I also attempted to test SVM but it was computationally taxing and inefficient, so I dropped it.

Here, I split my data into testing and training data using k-fold cross validation.
I will also be applying Grid Search to find the optimal parameters.

In [15]:
#from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
#from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
#from sklearn import tree
#from sklearn.svm import SVC

First I define my predictors and labels. (i.e X and y respectively)

In [16]:
cols = df.columns.values
X= df[cols[:8]]
y = df['y_class']

Now, applying K-fold cross vaidation to split the data into training and testing parts.

In [17]:
kf = KFold(n_splits=10, random_state=42, shuffle=False)
for train_index, test_index in kf.split(X):
    #print("Train Index: ", train_index, "\n")
    #print("Test Index: ", test_index)

    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]

Applying grid search on decision tree classifier to find the optimal parameters

**NOTE: I did not use the following three classifiers in the end but have run them to display the scores I received**

In [18]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
param_grid = {'max_depth': np.arange(1, 21),
              'min_samples_split' : range(10,500,20),
              'min_samples_leaf': [1, 5, 10, 20, 50, 100]}

clf = GridSearchCV(dtc,param_grid,cv=5)
clf.fit(X_train,y_train)
clf.best_params_
clf.score(X_test,y_test)

C:\Users\aleen\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.775

Doing the same for K nearest neighor Classifier

In [19]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()
param_grid = {'n_neighbors': np.arange(1, 25)}
clf1 = GridSearchCV(knn,param_grid,cv=5)
clf1.fit(X_train,y_train)
clf1.score(X_test,y_test)

C:\Users\aleen\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.55

And again for Logistic Regression Classifier

In [20]:
from sklearn.linear_model import LogisticRegression
grid={"C":np.logspace(-3,3,7), "penalty":["l2"]}
logreg=LogisticRegression(solver='newton-cg')
logreg_cv=GridSearchCV(logreg,grid,cv=5)
logreg_cv.fit(X_train,y_train)
logreg_cv.score(X_test,y_test)

C:\Users\aleen\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.6

Defining the classifiers with the optimal parameters

In [21]:
#dtc1=clf.best_estimator_
#logrg=logreg_cv.best_estimator_
#knn1=clf1.best_estimator_

And now putting them together into one ensemble classifier and fitting, then testing the data.

In [22]:
#from sklearn.ensemble import VotingClassifier
##create a dictionary of our models
#estimators=[('logref', rf1) ,('dtc', dtc1),('knn',knn1)]
#ensemble = VotingClassifier(estimators, voting='hard')
#ensemble.fit(X_train, y_train)
#ensemble.score(X_test, y_test)

An attempt was done without using K-fold cross validation with the same classifiers and a grid search, but the score was significantly lower i.e 0.7 as compared to 0.8
This proved to be a better approach and the previous attempts' code was not included to avoid cluttering.

At the end, I chose to use Random Forest Classifier, alone. I attempted my above code with random forests instead of logistic regression, but decided it was the most efficient on its' own and dropped the ensemble classifier. RFC it is not subject to overfitting, like its relative - decision tree. I came to the conclusion that it is the best choice for our given dataset and purpose.

In [37]:
rfc = RandomForestClassifier()
params_rf = {'n_estimators': [10, 50, 100, 200],'random_state':np.linspace(0,200,11,dtype=int)}
rf_gs = GridSearchCV(rfc, params_rf, cv=5)
rf_gs = rf_gs.fit(X_train, y_train)
#rf_gs.best_params_
rf_gs.score(X_test,y_test)

C:\Users\aleen\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.9

In [38]:
rf=rf_gs.best_estimator_
y_pred=rf.predict(X_test)
#The above line of code utilizes a predict function with our random forests classifier to give us our expected labels.
y_pred

array([1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1], dtype=int64)

In [39]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.85      0.89        20
           1       0.86      0.95      0.90        20

    accuracy                           0.90        40
   macro avg       0.90      0.90      0.90        40
weighted avg       0.90      0.90      0.90        40

